# 한국어 golden summary 추출

In [ ]:
with open("summaries.txt", "r") as f:
    lines = f.readlines()
    with open("kogolden.txt", "w") as ans:
        for line in lines:
            if line.startswith("GOLD"):
                ans.writelines(line[6:])

# 라이브러리 불러오기

In [ ]:
import urllib.request
import json

# papago api client 설정

> 발급 받은 api client id&secret을 json으로 저장

In [ ]:
with open("./client.json", "r") as file:
    client = json.load(file)

In [ ]:
client_id = client["client_id"]
client_secret = client["client_secret"]

# papago NMT api 함수 정의

In [ ]:
def mt(sentence):
    koText = urllib.parse.quote(sentence)
    data = "source=ko&target=en&text=" + koText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        json_result = json.loads(response_body.decode('utf-8'))
        enText = json_result["message"]["result"]["translatedText"]
        return enText
    else:
        print("Error Code:" + rescode)

# ko -> en translation

In [ ]:
cnt = 0

with open("kogolden.txt", "r") as ko:
    lines = ko.readlines()

    with open("engolden.txt", "a") as en:
        with open("engolden_preprocess.txt", "a") as pre:
            complete = en.readlines()
            start = len(complete)

            for line in lines[start:]:
                enText = mt(line)
                en.write(enText+"\n")
                pre.write("A painting with the vibe that " + enText+"\n")
                cnt += 1

print(cnt, "sentences translated in English and preprocessed successfully!")